<a href="https://colab.research.google.com/github/oburckard/MyRepo/blob/master/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np

#####AITKEN MANY DIMENSIONS - Correction #####

# Example:

def jacobian_example(xy):
    x, y = xy
    return [[1, 2],
            [2*x, 8*y]]

def function_example(xy):
    x, y = xy
    return [x + 2*y - 2, x**2 + 4*y**2 - 4]




# Another  exercise:
def function_exercise(xyz):
    x, y, z = xyz
    return [x + y + z - 3,
            x**2 + y**2 + z**2 - 5,
            np.exp(x) + x*y - x*z - 1]

def jacobian_exercise(xyz):
    x, y, z = xyz
    return [[1, 1, 1],
            [2*x, 2*y, 2*z],
            [np.exp(x) + y - z, x, -x]]

# slow convergence
def jacobian_slow(xy):
    x, y = xy
    return [[2*x, 3*3*y**2],
            [1, 1]]

def function_slow(xy):
    x, y = xy
    return [x**2 + 3*y**3 , x + y ]

# slow convergence 2
def jacobian_slow_2(xy):
    x, y = xy
    return [[2*x, 2*y],
            [1, 1]]

def function_slow_2(xy):
    x, y = xy
    return [x**2 + y**2 - 0.0000000000001, x + y ]

    
# iterative Newton's method
def iterative_newton(fun, x_init, jacobian):
    max_iter = 150
    epsilon = 1e-8

    x_last = x_init

    for k in range(max_iter):
        # Solve J(xn)*( xn+1 - xn ) = -F(xn):
        J = np.array(jacobian(x_last))
        F = np.array(fun(x_last))


        # inverted Jacobian
        Jinv = np.linalg.inv(J)
        #  x(n+1) = x(n) − J^(−1)(x(n))*F(x(n)).
        #  np.dot - array/vector multiplication
        diff = np.dot(Jinv,F)
        x_last = x_last - diff
        
        print(x_last)        

        # Stop condition:
        if np.linalg.norm(diff) < epsilon:
            print('convergence!, nre iter:', k )
            break

    else: # only if the for loop end 'naturally'
        print('not converged')

    return x_last


# iterative Newton's method with Aitken's acceleration
def iterative_aitken(fun, x_init, jacobian):
    max_iter = 50
    epsilon = 1e-8

    # Aitken's acceleration from http://www.zen89632.zen.co.uk/R/EM_Aitken/em_aitken.pdf

    loop_break = 0


    
    l = len(x_init)
    X = np.zeros([max_iter+l+2,l])
    X[0] = x_init

    for n in range(0, max_iter, l):
      if loop_break:
        break
      for k in range (0,l):
        # Solve J(xn)*( xn+1 - xn ) = -F(xn):
        J = np.array(jacobian(X[n+k]))
        F = np.array(fun(X[n+k]))


        # inverted Jacobian
        Jinv = np.linalg.inv(J)
        #  x(n+1) = x(n) − J^(−1)(x(n))*F(x(n)).
        #  np.dot - array/vector multiplication
        diff = np.dot(Jinv,F)
        X[n+k+1] = X[n+k] - diff

        
        print(X[n+k+1])   

        
      X1=np.zeros([l,l])
      X2=np.zeros([l,l])
      X3=np.zeros([l,l])

      for i in range(l):
          for j in range(l):
            X1[j,i] = X[n+k-1-i,j] 
            X2[j,i] = X[n+k-i,j] 
            X3[j,i] = X[n+k+1-i,j] 
        
      #print("X1 ",X1)
      #print("X2 ",X2)
      #print("X3 ",X3)
      #print("X2-X1 ",np.subtract(X2,X1)," det = ", np.linalg.det(np.subtract(X2,X1)))
      if abs( np.linalg.det(np.subtract(X2,X1)) ) > 1e-16:
           Lambda = np.dot(np.subtract(X3,X2),np.linalg.inv(np.subtract(X2,X1)))
           #print("Lambda = ",Lambda)
           XX1 = np.linalg.inv(np.subtract(np.identity(l),Lambda))
           XX2 = np.subtract(X[n+1],np.dot(Lambda,X[n]))
           XX  = np.dot(XX1, XX2)
           #print(" XX0 = ",np.subtract(np.identity(l),Lambda),np.identity(l),Lambda)
           #print(" XX1 = ",XX1)
           #print(" XX2 = ",XX2)
           print(" Without acceleration = ",X[n+k+1])
           print(" Aitken's accelerated = ",XX)
           X[n+k+1] = XX

          


      # Stop condition:
      if np.linalg.norm(diff) < epsilon:
            print('convergence!, nre iter:', n )
            X_out = X[n+k+1]
            loop_break = 1
            break

    else: # only if the for loop end 'naturally'
        print('not converged')
        X_out = X[n+k+1]

    return X_out





# slightly different implementation solving a set of linear equations
def iterative_newton_v2(fun, x_init, jacobian):
    max_iter = 50
    epsilon = 1e-8

    x_last = x_init

    for k in range(max_iter):
        # Solve J(xn)*( xn+1 - xn ) = -F(xn):
        J = np.array(jacobian(x_last))
        F = np.array(fun(x_last))

        diff = np.linalg.solve( J, -F )
        x_last = x_last + diff

        # Stop condition:
        if np.linalg.norm(diff) < epsilon:
            print('convergence!, nre iter:', k )
            break

    else: # only if the for loop end 'naturally'
        print('not converged')

    return x_last



# For the exercice:
#x_sol = iterative_newton(function_exercise, [2.0,1.0,2.0], jacobian_exercise)
#print('solution exercice:', x_sol )
#print('F(sol)', function_exercise(x_sol) )
print(" ")

# second version
#x_sol = iterative_newton_v2(function_exercise, [2.0,1.0,2.0], jacobian_exercise)
#print('solution exercise:', x_sol )
#print('F(sol)', function_exercise(x_sol) )
print(" ")

# slow convergence
x_sol = iterative_newton(function_slow, [2.0,1.0], jacobian_slow)
print('solution slow:', x_sol )
print('F(sol)', function_slow(x_sol) )
print(" ")

x_sol = iterative_aitken(function_slow, [2.0,1.0], jacobian_slow)
print('solution slow:', x_sol )
print('F(sol)', function_slow(x_sol) )
print(" ")

# slow convergence 2
x_sol = iterative_newton(function_slow_2, [2.0,1.0], jacobian_slow_2)
print('solution slow:', x_sol )
print('F(sol)', function_slow_2(x_sol) )
print(" ")

x_sol = iterative_aitken(function_slow_2, [2.0,1.0], jacobian_slow_2)
print('solution slow:', x_sol )
print('F(sol)', function_slow_2(x_sol) )
print(" ")




# For the example:
#x_sol = iterative_newton(function_example, [1.0,2.0], jacobian_example)
#print('solution example:', x_sol )
#print('F(sol)',function_example(x_sol) )
    

 
 
[-2.  2.]
[-1.3  1.3]
[-0.8350365  0.8350365]
[-0.52743867  0.52743867]
[-0.32556764  0.32556764]
[-0.19503289  0.19503289]
[-0.11271012  0.11271012]
[-0.06267652  0.06267652]
[-0.03363636  0.03363636]
[-0.01755518  0.01755518]
[-0.0089918  0.0089918]
[-0.00455418  0.00455418]
[-0.00229233  0.00229233]
[-0.00115007  0.00115007]
[-0.00057602  0.00057602]
[-0.00028826  0.00028826]
[-0.00014419  0.00014419]
[-7.21113751e-05  7.21113751e-05]
[-3.60595863e-05  3.60595863e-05]
[-1.80307682e-05  1.80307682e-05]
[-9.01562792e-06  9.01562792e-06]
[-4.50787492e-06  4.50787492e-06]
[-2.2539527e-06  2.2539527e-06]
[-1.12698016e-06  1.12698016e-06]
[-5.63491033e-07  5.63491033e-07]
[-2.81745755e-07  2.81745755e-07]
[-1.40872937e-07  1.40872937e-07]
[-7.04364833e-08  7.04364833e-08]
[-3.52182454e-08  3.52182454e-08]
[-1.76091236e-08  1.76091236e-08]
[-8.80456204e-09  8.80456204e-09]
[-4.40228108e-09  4.40228108e-09]
convergence!, nre iter: 31
solution slow: [-4.40228108e-09  4.40228108e-09]
F(so

4*x1 − x2 + x3 = 8

2*x1 + 5*x2 + 2*x3 = 3

x1 + 2*x2 + 4*x3 = 11

=> 4*x1 − x2 + x3 = 8

   11*x2 + 3*x3 = -2

   9*x2 + 15*x3 = 36

=> 4*x1 − x2 + x3 = 8

   46*x2 = -46

   9*x2 + 15*x3 = 36

=> 4*x1 − x2 + x3 = 8

   x2 = -1
   x3 = 3


=> x1 = 1

   x2 = -1

   x3 = 3

------------------------

4*x1 − x2 + 2*x3 = 9

2*x1 + 4*x2 − x3 = −5

x1 + x2 − 3*x3 = −9

=> -5*x2 + 14*x3 = 45

   2*x2 + 5*x3 = 13

   x1 + x2 − 3*x3 = −9

=> -5*x2 + 14*x3 = 45 

   (53/5) * x3 = 31

   x1 + x2 − 3*x3 = −9

=> x3 = 155/53

   x2 = -43/53
   
   x1 = 31/53   

In [0]:
# Gaussian elimination

import scipy 
import scipy.linalg
import numpy as np
import copy

def linearsolver(A,b):
  n = len(A)
  M = A

  i = 0
  for x in M:
   x.append(b[i])
   i += 1


  for k in range(n):
   # first pivoting
   for i in range(k,n):
     if abs(M[i][k]) > abs(M[k][k]):
        M[k], M[i] = M[i],M[k]
     else:
        pass
   
   # Gaussian elimination in loop 
   for j in range(k+1,n):
       q = float(M[j][k]) / M[k][k]
       for m in range(k, n+1):
          # M_jm = M_jm - M_jk / M_kk *M_km
          M[j][m] -=  q * M[k][m]

  x = [0 for i in range(n)]       # x = [0, 0, .... , 0]

  # calculating x
  x[n-1] =float(M[n-1][n])/M[n-1][n-1]
  for i in range (n-1,-1,-1):
    z = 0
    for j in range(i+1,n):
        z = z  + float(M[i][j])*x[j]
    x[i] = float(M[i][n] - z)/M[i][i]
  return x


#A = [[2, 1, 1, 0], [4, 3, 3, 1], [8, 7, 9, 5], [6, 7, 9, 8]]
#b = [1, 2, 3, 4]

A = [[4, -1, 1], [2, 5, 2], [1, 2, 4]]
b = [8, 3, 11]

#A = [[4, -1, 2], [2, 4, -1], [1, 1, -3]]
#b = [9, -5, -9]

A1 = copy.deepcopy(A)
x = linearsolver(A1, b)

print("Result with our linearsolver:")
print(np.array(A)," * x = ",np.array(b))
print("Solution:")
print(x)




x = scipy.linalg.solve(A,b)

print(" ")
print("Result with linalg.solve:")
print(np.array(A)," * x = ",np.array(b))
print("Solution:")
print(x)

Result with our linearsolver:
[[ 4 -1  1]
 [ 2  5  2]
 [ 1  2  4]]  * x =  [ 8  3 11]
Solution:
[1.0, -1.0, 3.0]
 
Result with linalg.solve:
[[ 4 -1  1]
 [ 2  5  2]
 [ 1  2  4]]  * x =  [ 8  3 11]
Solution:
[ 1. -1.  3.]


In [0]:
# LU decomposition

import numpy as np

def LU_partial_decomposition(matrix,b):
    n, m = matrix.shape
    P = np.identity(n)
    L = np.identity(n)
    U = matrix.copy()
    bb = b.copy()
    PF = np.identity(n)
    LF = np.zeros((n,n))
    for k in range(0, n - 1):
        index = np.argmax(abs(U[k:,k]))
        index = index + k 
        
        # here we do pivoting
        if index != k:
            P = np.identity(n)
            P[[index,k],k:n] = P[[k,index],k:n]
            U[[index,k],k:n] = U[[k,index],k:n]
            bb[k] = b[index]
            bb[index] = b[k]
            b = bb.copy()
            PF = np.dot(P,PF)
            LF = np.dot(P,LF)
            
        # and here decomposition    
        L = np.identity(n)
        for j in range(k+1,n):
            L[j,k] = -(U[j,k] / U[k,k])
            LF[j,k] = (U[j,k] / U[k,k])
        U = np.dot(L,U)
    np.fill_diagonal(LF, 1)
  
    return PF, LF, U, bb


def solve_Ly(L,b):
  n, n = L.shape
  y = np.zeros(n)
  y[0] = b[0]/L[0,0]
  for k in range(1,n):
    y[k] = b[k]
    for i in range(0,k):
      y[k] = y[k] - L[k,i] * y[i]
    y[k] = y[k] / L[k,k]
    
  return y  



def solve_Ux(U,y):
  n, n = U.shape
  x = np.zeros(n)
  x[n-1] = y[n-1]/U[n-1,n-1]
  for k in range(n-2,-1,-1):
    x[k] = y[k]
    for i in range(n-1,k,-1):
       x[k] = x[k] - U[k,i] * x[i]
    x[k] = x[k] / U[k,k]

  return x 
      


# Usage
#A = [[2, 1, 1, 0], [4, 3, 3, 1], [8, 7, 9, 5], [6, 7, 9, 8]]
#b = [1, 2, 3, 4]
A = [[1,2,1], [1,-2,2], [2,12,-2]]
b = [0,4,4]
#A = [[4, -1, 1], [2, 5, 2], [1, 2, 4]]
#b = [8, 3, 11]
#A = [[4, -1, 2], [2, 4, -1], [1, 1, -3]]
#b = [9, -5, -9]
A = np.array(A)
P1, L1, U1, bb = LU_partial_decomposition(A,b)
y = solve_Ly(L1,bb)
x = solve_Ux(U1,y)

# Scipy
import scipy
import scipy.linalg
#LU decomposition using linalg.lu library function
P2, L2, U2 = scipy.linalg.lu(A) # P2 = P2.T  

print("Input array:")
print(A)
print(" ")
print("Result. Pivoting array:")
print(P1)
print("L and U arrays:")
print(L1,"\n", U1)
print(" ")
print("Result using linalg.lu function. Pivoting array:")
print(P2.T)  # transponed array
print("L and U arrays:")
print(L2,"\n", U2)
print('  ')
print("Results :")
print("y:", y)
print("x :",x)

Input array:
[[ 1  2  1]
 [ 1 -2  2]
 [ 2 12 -2]]
 
Result. Pivoting array:
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
L and U arrays:
[[1.  0.  0. ]
 [0.5 1.  0. ]
 [0.5 0.5 1. ]] 
 [[ 2.  12.  -2. ]
 [ 0.  -8.   3. ]
 [ 0.   0.   0.5]]
 
Result using linalg.lu function. Pivoting array:
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
L and U arrays:
[[1.  0.  0. ]
 [0.5 1.  0. ]
 [0.5 0.5 1. ]] 
 [[ 2.  12.  -2. ]
 [ 0.  -8.   3. ]
 [ 0.   0.   0.5]]
  
Results :
y: [ 4.  2. -3.]
x : [11.  -2.5 -6. ]
